@[TOC]

## 库

In [1]:
import os
import csv
import requests
import xlwt
import  re
import json
import time

### 配置

In [2]:
#根据个人浏览器信息进行修改
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
    ,
    'Cookie': '_T_WM=32842944849; SCF=AseBZ55gn3zgZoQoehy2jTSXuGHcU-4AaGTFxF6GwDwlZ-LBNXr5j2p4gt7YXW9sHGcUTak396JzBhl3dhPTFbU.; SUB=_2A25OCGzYDeRhGeNJ6lUT9SfMyjSIHXVt83SQrDV6PUNbktB-LUHykW1NS-TN0QqN08t6J65jfPmWROlEux_y-zDe; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5D8615S5j4Ho8woelwSafF5JpX5KMhUgL.Fo-NeKMESK.7eKn2dJLoI0YLxKBLBonL1h5LxK-L1-zLBKzLxKMLBoeL1-qLxKML1-2L1hBLxK-LBo5L12qLxK.LB.zL1K2LxK-LB-BL1K5t; SSOLoginState=1661738120; ALF=1664330120; WEIBOCN_FROM=1110006030; MLOGIN=1; XSRF-TOKEN=2c036a'
    #'ALF=1581501545; _T_WM=67706607048; H5_wentry=H5; backURL=https%3A%2F%2Fm.weibo.cn%2Fapi%2Fcomments%2Fshow%3Fid%3DIr5j4iRXW%26page%3D3; XSRF-TOKEN=11216a; WEIBOCN_FROM=1110006030; MLOGIN=1; SSOLoginState=1580006602; SCF=AqQddu0eGCw6Wh1xPsTyigWBFJH-P0ACsyLUFzNakys5zFt06rZeA1gEI0iP7HfWxZntbpMr8WTWhrxEdSVGB58.; SUB=_2A25zKIyaDeRhGeNP41UT9yjIyj6IHXVQ0hTSrDV6PUJbktAKLRL-kW1NTk4PgHLYgtoeuxFzuGDIDcybzoEoXvq9; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5q.Hx0pIs7PKpACzdnFYSZ5JpX5KzhUgL.Fo-p1hMES0qXeKz2dJLoIpUeBc8EdFH8SC-4BbHFSFH81F-RSF-4Sntt; SUHB=0IIlrfWMMkVsTI; M_WEIBOCN_PARAMS=uicode%3D20000174'
}

In [3]:
#文件保存地址
addrRoot='./dataset/'

In [4]:
#是否获取转发者具体个人信息
getConcreteInfoList=False#True#
isLogin=False#True

#是否登入采集个人信息

#无信息打印字符
infoNoExistStr='未知'


In [5]:
#是否处理微博文本内容
processText = False#True

### 构造表格，采集数据内容（修改这里获取想要的信息）

In [6]:
#获取个人具体信息范围、排列
infoRangeDict={
'性别':True,
'所在地':True,
    
'生日':False,
'家乡':False,
'公司':True,
'大学':True,
    
'昵称':False,
'简介':False,
'注册时间':False,
'阳光信用':False,
    
    #若无信息显示
'infoNoExist':'未知'
}



In [7]:
#获取博文信息范围、排列
blogRangeDict={
'visible': False,#{type: 0, list_id: 0}
    
'created_at': True,#"20分钟前"
    
'id': False,#"4466073829119710"
'idstr': False,#"4466073829119710"
'mid': False,#"4466073829119710"
'can_edit': False,#false
'show_additional_indication': False,#0
    
'text': True,#"【情况通报】2019年12月31日，武汉市卫健部门发布关于肺炎疫情的情况通报。
    
'textLength': False,#452
'source': False,#"360安全浏览器"
'favorited': False,#false
'pic_types': False,#""
'is_paid': False,#false
'mblog_vip_type': False,#0
'user': False,#{id: 2418542712, screen_name: "平安武汉",…}
    
'reposts_count': True,#1035
'comments_count': True,#1886
'attitudes_count': True,#7508
    
'pending_approval_count': False,#0
'isLongText': False,#true
'reward_exhibition_type':False,# 0
'hide_flag': False,#0
'mblogtype': False,#0
'more_info_type': False,#0
'cardid': False,#"star_11247_common"
'content_auth': False,#0
'pic_num': False,#0
    
#若无相关信息，则显示：
'infoNoExist':'未知'
}

In [8]:
#获取博主信息范围、排列
userRangeDict={
'id':True,# 1323527941
'screen_name': True,#"Vista看天下"
    
'profile_image_url': False,#"https://tva2.sinaimg.cn/crop.0.0.180.180.180/4ee36f05jw1e8qgp5bmzyj2050050aa8.jpg?KID=imgbed,tva&Expires=1580290462&ssig=xPIoKDRR56"
'profile_url':False,# "https://m.weibo.cn/u/1323527941?uid=1323527941&luicode=10000011&lfid=1076031323527941"
'statuses_count': False,#微博数 77256
'verified': False,#true
'verified_type':False,# 3
'verified_type_ext': False,#0
'verified_reason': False,#"《Vista看天下》官方微博"
'close_blue_v': False,#false
    
'description': True,#"一个有趣的蓝V"
'gender': True,# "m"
    
'mbtype': False,#12
'urank': False,#48
'mbrank': False,#6
'follow_me':False,# false
'following':False,# false
    
'followers_count': True,#19657897
'follow_count': True,#1809
    
'cover_image_phone': False,#"https://tva1.sinaimg.cn/crop.0.0.640.640.640/549d0121tw1egm1kjly3jj20hs0hsq4f.jpg"
'avatar_hd': False,#"https://ww2.sinaimg.cn/orj480/4ee36f05jw1e8qgp5bmzyj2050050aa8.jpg"
'like': False,#false
'like_me': False,#false
'badge': False,#{enterprise: 1, gongyi_level: 1, bind_taobao: 1, dzwbqlx_2016: 1, follow_whitelist_video: 1,…}
    
#若无信息显示
'infoNoExist':'未知'
}


### 文件命名

In [9]:
def addrFile(tweeter,suffix):
    path=addrRoot+str(tweeter)+'/'
    if os.path.exists(path) is False:
         os.makedirs(path)
    address=path+tweeter+suffix+'.csv'
    return address  

### 生成信息标题

In [10]:
def getInfoTitle(Dict,prefix):
    titleList=[]
    for item in Dict:
        if(Dict.get(item) is True):
            titleList.append(prefix+item)
    return (titleList)

## 工具类，用来去除爬取的正文中一些不需要的链接、标签等

In [11]:
#工具类，用来去除爬取的正文中一些不需要的链接、标签等
class Tool:
    deleteImg = re.compile('<img.*?>')
    newLine =re.compile('<tr>|<div>|</tr>|</div>')
    deleteAite = re.compile('//.*?:')
    deleteAddr = re.compile('<a.*?>.*?</a>|<a href='+'\'https:')
    deleteTag = re.compile('<.*?>')
    deleteWord = re.compile('回复@|回覆@|回覆|回复')
 
    @classmethod
    def replace(cls,x):
        x = re.sub(cls.deleteWord,'',x)
        x = re.sub(cls.deleteImg,'',x)
        x = re.sub(cls.deleteAite,'',x)
        x = re.sub(cls.deleteAddr, '', x)
        x = re.sub(cls.newLine,'',x)
        x = re.sub(cls.deleteTag,'',x)
        return x.strip()

### 构造微博转发信息的url

In [12]:
def repostURL(id,pages):
    urls=[]
    for page in pages:
        if page != 0:
            urls+=['https://m.weibo.cn/api/statuses/repostTimeline?id='+str(id)+'&page='+str(page)]
    print(urls)
    return urls   

In [13]:
#将字典类型的信息格式传递为需要的信息列表
def getInfoList(infoDict,rangeDict):
    infoList=[]
    for item in rangeDict:
        if rangeDict.get(item) is True:
            content=infoDict.get(item)
            if content is not None:
                #处理微博文本内容  
                if item =='text':
                    if processText is True:
                        content=Tool.replace(content)
                infoList.append(content)      
            else:
                infoList.append(rangeDict['infoNoExist'])
    return infoList

### 爬取所有转发信息

In [14]:
###在已有的一系列urls中进行操作
###筛选出微博转发内容进行操作
def reRatio(urls,csvWriter):
    
    notEnd= True
    
    retweetBlogTitle=getInfoTitle(blogRangeDict,'转发')#转发博文信息标题
    retweetUserTitle=getInfoTitle(userRangeDict,'转发')#转发博主信息标题
    
    
    infoTitle=getInfoTitle(infoRangeDict,'')#原文博主个人主页信息标题
    
    #写表格的标题
    if getConcreteInfoList is True:       
        csvWriter.writerow(retweetBlogTitle+retweetUserTitle+infoTitle)        
    else:
        csvWriter.writerow(retweetBlogTitle+retweetUserTitle)
        
    for url in urls:        
        
        response = requests.get(url,headers=headers)
        txt=response.text
        # print(drillInfo(txt))
        resjson = json.loads(response.text)    
        state=resjson['ok']
        #结束最后
        if(state==0):
            notEnd=False
            break
        
        cards=resjson['data']['data']      
        
        #print(cards)
        
        #结束最后
        if(len(cards)==1):
            notEnd=False
            break
        #遍历一个页面的所有微博    
        for card in cards:
            try:
                #转发博文与博主信息
                retweetBlogInfoDict=card 
                retweetUserInfoDict=retweetBlogInfoDict['user']      
                
                #构造填入csv文件数据列表
                infoList=[]                            

                #转发博文数据
                retweetBlogInfoList=getInfoList(retweetBlogInfoDict,blogRangeDict)               
                infoList+=retweetBlogInfoList   
                
                #转发博主数据
                ##默认已知
                retweetUserInfoList=getInfoList(retweetUserInfoDict,userRangeDict)               
                infoList+=retweetUserInfoList  
                
                                  

                if getConcreteInfoList is True:
                    infoDict=getInfo(isLogin,originUserID)
                    otherInfoList=getInfoList(infoDict,infoRangeDict)      
                    infoList+=otherInfoList                          
                #print(infoList)               
                #保存数据至csv
                csvWriter.writerow(infoList)                       

                #不断获取该博主对的影响力
                #break
            except:
                pass            
        #延时，防止反爬
        time.sleep(3)
        
    return notEnd

In [15]:
url = 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=2'
response = requests.get(url,headers=headers)
txt=response.text
print(txt)

{"ok":1,"msg":"\u6570\u636e\u83b7\u53d6\u6210\u529f","data":{"data":[{"visible":{"type":0,"list_id":0},"created_at":"Fri Sep 02 09:06:57 +0800 2022","id":"4809177874302342","mid":"4809177874302342","can_edit":false,"show_additional_indication":0,"text":"\u8f6c\u53d1\u5fae\u535a","source":"\u5fae\u535a\u641c\u7d22","favorited":false,"pic_ids":[],"is_paid":false,"mblog_vip_type":0,"user":{"id":3108071821,"screen_name":"\u8d5b\u535a\u4e00\u7ea7\u51b2\u6d6a\u9009\u624b","profile_image_url":"https:\/\/tvax2.sinaimg.cn\/crop.0.0.1024.1024.180\/b941698dly8h562fi4i6dj20sg0sgdgs.jpg?KID=imgbed,tva&Expires=1662091661&ssig=Bwl80Y7wns","profile_url":"https:\/\/m.weibo.cn\/u\/3108071821?uid=3108071821","statuses_count":3724,"verified":false,"verified_type":-1,"close_blue_v":false,"description":"\u9547\u9b42","gender":"f","mbtype":2,"urank":27,"mbrank":1,"follow_me":false,"following":false,"follow_count":281,"followers_count":"131","followers_count_str":"131","cover_image_phone":"https:\/\/tva1.sina

### 获取个人主页中信息

In [16]:
def drillInfo(txt):
    keyInfo={}
    
    try:  
        resjson = json.loads(txt)  
        infodata = resjson.get('data')
        cards = infodata.get('cards')
        for l in range(0,len(cards)):
            temp = cards[l]
            card_group = temp.get('card_group')            
            #判断获取信息类型                     
            for card in card_group:                
                #将信息传入字典
                name=card.get('item_name')
                if name is not None:
                    content=card.get('item_content')
                    keyInfo[name]=content 
    except:
        pass
    return keyInfo

###  构建通过id访问个人主页的url

In [17]:
def infoUrl(id):
    url = "https://m.weibo.cn/api/container/getIndex?containerid=230283"+str(id)+"_-_INFO" 
    return url

## 爬取某id博主的个人信息

In [18]:
def getInfo(state,id):
    
    address=addrRoot+'info/'+str(state)+'id'+str(id)+'.txt'
    path=addrRoot+'info/'
    if os.path.exists(path) is False:
        os.makedirs(path)
    try:
        #已有文件
        if(os.path.exists(address)==True):
            fp = open(address,'r',encoding='utf-16')
            txt=fp.read()
            info=drillInfo(txt)
            fp.close()
        else:  
            fp = open(address,'w+',encoding='utf-16')
            url=infoUrl(id)
            if state is True:
                response = requests.get(url,headers=headers)
            else:
                response = requests.get(url)
            txt=response.text
            fp.write(response.text)        
            info=drillInfo(txt)
            fp.close()
    except:
        info=-1     
    
    return info

In [19]:
def getExatInfo(item,state,id):
    info=getInfo(state,id)
    content=info.get(item)
    if content is not None:
        return content
    else:
        return infoNoExistStr        

In [20]:
###  构造热门界面访问

In [21]:
def downloadData(id):
    #tweeter=getExatInfo('昵称',2,int(id))
    tweeter='kobe'
    batch=0
    
    
    while(1):
        ranges=range(20*batch,20*(batch+1))

        fileAddr=addrFile(tweeter,'batch'+str(batch))
        if os.path.exists(fileAddr) is True:
            print(fileAddr+'已存在，跳过采集')                
        else:
            print('文件将写入：'+fileAddr)
            fp = open(fileAddr,'w+',newline='',encoding='utf-16')
            writer=csv.writer(fp)
            if reRatio(repostURL(id,ranges),writer) is False:
                fp.close()
                break

            fp.close()
            print('第'+str(batch)+'批数据已记录完毕')
        batch+=1        
    

In [22]:
id=input('博文id：')
#4102228300324979
#4466810701697847武汉肺炎
#4465738137650546
downloadData(id)

./dataset/kobe/kobebatch0.csv已存在，跳过采集
./dataset/kobe/kobebatch1.csv已存在，跳过采集
./dataset/kobe/kobebatch2.csv已存在，跳过采集
./dataset/kobe/kobebatch3.csv已存在，跳过采集
./dataset/kobe/kobebatch4.csv已存在，跳过采集
./dataset/kobe/kobebatch5.csv已存在，跳过采集
./dataset/kobe/kobebatch6.csv已存在，跳过采集
./dataset/kobe/kobebatch7.csv已存在，跳过采集
文件将写入：./dataset/kobe/kobebatch8.csv
['https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=160', 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=161', 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=162', 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=163', 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=164', 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=165', 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=166', 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=167', 'https://m.weibo.cn/api/statuses/repostTimeline?id=M3JCVtCDV&page=168', 'https://m.

In [ ]:
fp.close()

In [ ]:
print(resjson)